# Train Music Transformer
Since Fri. May 13th, 2022

Set up training in colab


## Setup


### Ipython



In [1]:
%load_ext autoreload
%autoreload 2



### Colab



In [2]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets
    ! pip3 install stefutils

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from musicnlp.util import *

ic(u.proj_dir, u.pkg_nm)



ic| u.proj_dir: 'Symbolic-Music-Generation', u.pkg_nm: 'musicnlp'


('Symbolic-Music-Generation', 'musicnlp')

### code

In [3]:
import transformers

from musicnlp.trainer import train

transformers.set_seed(sconfig('random-seed'))



## Prep for training


In [4]:
md_nm = 'transf-xl'
# md_sz = 'debug'
# md_sz = 'debug-large'
md_sz = 'base'
# n = 32
n = None
ic(md_sz, n)

augment_key = False
max_length = 512   # TODO: short length to test for now
num_train_epochs = 4  # TODO

pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-04'
mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-28'
lmd = 'musicnlp music extraction, dnm=LMD, n=176640, meta={mode=melody, prec=5, th=1}, 2022-05-27_15-23-20'
dnms = [pop, mst, lmd]
my_train_args = dict(
    augment_key=augment_key,
    save_epochs=2
)
train_args = dict()

with_tqdm = True
if with_tqdm:
    my_train_args.update(dict(tqdm=True, logging_strategy='epoch'))
if 'debug' not in md_sz:
    train_args = dict(
        per_device_train_batch_size=16,
        num_train_epochs=num_train_epochs
    )
mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, dataset_names=dnms, n_sample=n,
    train_args=train_args, my_train_args=my_train_args
)



## Train


Check log and tensorboard files written



In [5]:
ignore_keys_for_eval = ['losses', 'mems', 'hidden_states', 'attentions']
trainer.train(ignore_keys_for_eval=ignore_keys_for_eval)
trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))



2022-05-23 22:05:36|[MyTransfoXLLMHeadModel Training]::on_train_begin::train_util_wrap.py:172:INFO: Training started with model {model name: MyTransfoXLLMHeadModel, max length: 128, n_layer: 4, hidden_size: 128, ff_size: 512, parameter_count: 917.2K}, {
    "vocab_size": 462,
    "cutoffs": [],
    "tie_projs": [
        false
    ],
    "d_model": 128,
    "d_embed": 128,
    "d_head": 16,
    "d_inner": 512,
    "div_val": 1,
    "pre_lnorm": false,
    "n_layer": 4,
    "n_head": 8,
    "mem_len": 64,
    "same_length": true,
    "attn_type": 0,
    "clamp_len": 1000,
    "sample_softmax": -1,
    "adaptive": true,
    "dropout": 0.1,
    "dropatt": 0.0,
    "untie_r": true,
    "init": "normal",
    "init_range": 0.01,
    "proj_init_std": 0.01,
    "init_std": 0.02,
    "layer_norm_epsilon": 1e-05,
    "return_dict": true,
    "output_hidden_states": false,
    "output_attentions": false,
    "torchscript": false,
    "torch_dtype": null,
    "use_bfloat16": false,
    "pruned_hea

## Sanity check



In [6]:
ic(trainer.args.output_dir)
ic(os.listdir(trainer.args.output_dir))



ic| trainer.args.output_dir: ('/Users/stefanhg/Documents/UMich/Research/Music with '
                              'NLP/Symbolic-Music-Generation/models/transf-xl/2022-05-23_22-05-35')
ic| os.listdir(trainer.args.output_dir): ['md={nm=MyTransfoXLLMHeadModel, l=128, n_l=4, hd_sz=128, ff_sz=512, '
                                          'n_param=917.2K}, n=32, a=0.001, bsz=8, n_ep=16.log',
                                          'checkpoint-8',
                                          'tb - md={nm=MyTransfoXLLMHeadModel, l=128, n_l=4, hd_sz=128, ff_sz=512, '
                                          'n_param=917.2K}, n=32, a=0.001, bsz=8, n_ep=16',
                                          'checkpoint-16',
                                          'checkpoint-56',
                                          'trained',
                                          'checkpoint-32',
                                          'checkpoint-40',
                                          'checkpoi

['md={nm=MyTransfoXLLMHeadModel, l=128, n_l=4, hd_sz=128, ff_sz=512, n_param=917.2K}, n=32, a=0.001, bsz=8, n_ep=16.log',
 'checkpoint-8',
 'tb - md={nm=MyTransfoXLLMHeadModel, l=128, n_l=4, hd_sz=128, ff_sz=512, n_param=917.2K}, n=32, a=0.001, bsz=8, n_ep=16',
 'checkpoint-16',
 'checkpoint-56',
 'trained',
 'checkpoint-32',
 'checkpoint-40',
 'checkpoint-24',
 'checkpoint-48',
 'checkpoint-64']